In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('ggplot')

In [ ]:
data = pd.read_csv('../input/heart-disease/heart.csv')

In [ ]:
data.head()

In [ ]:
data.isnull().sum()

In [ ]:
data.dtypes

## EDA

In [ ]:
data.hist(figsize=(10,8))
plt.tight_layout()
plt.show()

In [ ]:
# distribution plot for numeric variables

In [ ]:
fig, ax = plt.subplots(5,1, figsize=(11,10))
sns.histplot(data['age'], ax=ax[0], kde=True)
sns.histplot(data['trestbps'], ax=ax[1], kde=True)
sns.histplot(data['chol'], ax=ax[2], kde=True)
sns.histplot(data['thalach'], ax=ax[3], kde=True)
sns.histplot(data['oldpeak'], ax=ax[4], kde=True)

In [ ]:
# correlation and heatmap

In [ ]:
corr_matrix = data.corr()
mask = np.array(corr_matrix)
mask[np.tril_indices_from(mask, 0)] = False

In [ ]:
plt.figure(figsize=(11,9))
sns.heatmap(corr_matrix, mask=mask, annot=True, 
           square=True, cmap='viridis')
plt.tight_layout()
plt.xticks(rotation=45)
plt.show()

In [ ]:
#Data preprocessing

In [ ]:
cat_variables = ['cp', 'restecg', 'slope', 'thal']
X = data.iloc[:,:-1].copy()
y = data['target']

In [ ]:
X_enc = pd.get_dummies(X, columns=cat_variables)

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import plot_confusion_matrix

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_enc, y, test_size=0.33, random_state=101)

In [ ]:
clf = DecisionTreeClassifier(random_state=101)
clf = clf.fit(X_train, y_train)

In [ ]:
plot_confusion_matrix(clf, X_test, y_test)
plt.grid(())

In [ ]:
plt.figure(figsize=(20,15))
plot_tree(clf, 
          rounded=True,
         filled=True,
         feature_names=X_enc.columns)
plt.show()

`pruning trees

In [ ]:
path = clf.cost_complexity_pruning_path(X_train, y_train)

In [ ]:
ccp_alphas = path['ccp_alphas'][:-1]

In [ ]:
clfs = []

for alpha in ccp_alphas:
    dt = DecisionTreeClassifier(random_state=101, ccp_alpha=alpha)
    dt = dt.fit(X_train, y_train)
    clfs.append(dt)

In [ ]:
# calculate scores for both train and set set
train_scores = [dt.score(X_train, y_train) for dt in clfs]
test_scores = [dt.score(X_test, y_test) for dt in clfs] 

In [ ]:
plt.figure(figsize=(11,7))
plt.plot(ccp_alphas, train_scores, label='train set', color='r', marker='o', ls= '--', drawstyle='steps-post')
plt.plot(ccp_alphas, test_scores, label='test set', color='g',marker='o', ls= '--', drawstyle='steps-post')
plt.legend()

In [ ]:
# Choosing alpha 0.017 as it leads to higher accuracy

In [ ]:
# Using cross validation to test 0.017 (alpha) 

In [ ]:
clf = DecisionTreeClassifier(random_state=101, ccp_alpha=0.017)

In [ ]:
scores = cross_val_score(clf, X_train, y_train, cv=5)
df_scores = pd.DataFrame(data={'tree': range(1,6),
                               'accuracy': scores})

In [ ]:
df_scores.head()

In [ ]:
plt.figure(figsize=(10,8))
df_scores.plot(x='tree', y='accuracy', marker='o', ls='--')

In [ ]:
# As the accuracy is quite sensitive to different datasets, so their mean and std are calculated

alpha_cross = []

for alpha in ccp_alphas:
    dt = DecisionTreeClassifier(random_state=101, ccp_alpha=alpha)
    scores = cross_val_score(dt, X_train, y_train, cv=5)
    alpha_cross.append([alpha, np.mean(scores), np.std(scores)])

df_cross = pd.DataFrame(alpha_cross, 
                        columns=['ccp_alpha', 'mean_accuracy', 'std_accuracy'])

In [ ]:
df_cross.head()

In [ ]:
plt.figure(figsize=(11,9))
df_cross.plot(x='ccp_alpha',
              y='mean_accuracy',
              yerr='std_accuracy', 
              marker='o',
              ls='--')

In [ ]:
# Take the best value
df_cross.sort_values(by='mean_accuracy', ascending=False).head(3)

In [ ]:
final_alpha = df_cross.iloc[15,0]
print("The ideal alpha for this dt model is %3.5f" % final_alpha)

In [ ]:
# Buiding the final model

final_clf = DecisionTreeClassifier(random_state=101, ccp_alpha=final_alpha)
fianl_clf = final_clf.fit(X_train, y_train)

In [ ]:
# plot confusion matrix
plot_confusion_matrix(final_clf, X_test, y_test)
plt.grid(())

In [ ]:
# It is better than previous one!! 
# Finished !! 
